In [17]:
import warnings
warnings.filterwarnings("ignore")
import cv2
import matplotlib.pyplot as plt
import numpy as np
import time

### Vehicle detection on static Camera

In [24]:
#---tutorial:https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_gui/py_video_display/py_video_display.html
#keypoint : The video is read frame by frame and each frame
#can be treated as an image and hence all cv2 image processing can be applied on the frames of the videos
cap = cv2.VideoCapture(r'E:\Video\video1.mp4')
#---detect the object using background subtraction-----#
object_detector = cv2.createBackgroundSubtractorMOG2(history=200,varThreshold=90)

while(cap.isOpened()):
    stime= time.time()
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        mask = object_detector.apply(frame)
        _,mask = cv2.threshold(mask,254,255,cv2.THRESH_BINARY) #---will retain only white pixels in mask while removing all dark ot grey pixels i.e all the pixels 254 and below will be removed
        contours,_ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        #----removing the smaller object contours----#
        for cnt in contours:
            area = cv2.contourArea(cnt)
            if area > 200:
                #cv2.drawContours(frame,[cnt],-1,(0,255,0),2)
                x,y,w,h = cv2.boundingRect(cnt)
                cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.imshow('original',frame)
        cv2.imshow('mask',mask)
        if cv2.waitKey(35) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


<li>We used background subtraction with history value 200 since amount of vehicles is high and we used varThreshold =90 to improve the accuracy of our prediction.
<li>After we used background subtraction we were able to easily identify moving objects since they were shown as white while most of the static objects were black

![title](BackgroundSubtraction.JPG)

<li>But in the above mask we could still observe a few small white objects that are noise in our mask
<li>In order to reduce them we can add thresholding that will retain only white pixels in mask while removing all dark or grey pixels i.e all the pixels 254 and below will be removed. Below is the image after thresholding that shows some improvement

![title](AfterAddingThreshold.JPG)

<li>Now we generate contours and will plot the contours that have area denser than 200 pixels

![title](ContourAreaLessThan200px.JPG)

<li>The vehicles are now getting identified but the bounding boxes are not uniform. Let us build the red bounding box around the vehicle

![title](vehicle_capture_on_static_camera.JPG)

### Vehicle detection on non static camera(using Classifier)

In [22]:
cap = cv2.VideoCapture(r'E:\Video\video2.mp4')
car_cascade = cv2.CascadeClassifier('cars.xml') 
while(cap.isOpened()):
    ret, frame = cap.read() 
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
        cars = car_cascade.detectMultiScale(gray, 1.1, 1) 
        for (x,y,w,h) in cars: 
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.imshow('original',frame)
        if cv2.waitKey(35) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

On non static camera

![title](vehicle_capture_non_static_camera.JPG)